In [2]:
import torch
import bock
from bock import SingleTaskGP, ExpectedImprovement, bo_step, CustomCylindricalGP
from IPython.display import clear_output


In [3]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from torch.nn import MSELoss
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import copy
from IPython.display import display, Math, Latex

import Formula
import functions
import importlib
from hessian import hessian
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))

device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")

In [4]:
def opt_func(X):
    return X.norm(dim=1)

In [11]:
# dimensionality of the problem
n_dim = 5
# number of init measurements
num_init = 20
# size of search space
x_step = 5.
# center of search space
center = 0.7

borders = []
for i in range(n_dim):
    borders.append((center - x_step, center + x_step))
borders = torch.tensor(borders).float()
soboleng = torch.quasirandom.SobolEngine(dimension=n_dim)
X = soboleng.draw(num_init).float()
print(X)
X = X * (borders[:, 1] - borders[:, 0]) + borders[:, 0]
y = opt_func(X)

tensor([[0.5000, 0.5000, 0.5000, 0.5000, 0.5000],
        [0.7500, 0.2500, 0.7500, 0.2500, 0.7500],
        [0.2500, 0.7500, 0.2500, 0.7500, 0.2500],
        [0.3750, 0.3750, 0.6250, 0.1250, 0.8750],
        [0.8750, 0.8750, 0.1250, 0.6250, 0.3750],
        [0.6250, 0.1250, 0.3750, 0.3750, 0.1250],
        [0.1250, 0.6250, 0.8750, 0.8750, 0.6250],
        [0.1875, 0.3125, 0.3125, 0.6875, 0.5625],
        [0.6875, 0.8125, 0.8125, 0.1875, 0.0625],
        [0.9375, 0.0625, 0.5625, 0.9375, 0.3125],
        [0.4375, 0.5625, 0.0625, 0.4375, 0.8125],
        [0.3125, 0.1875, 0.9375, 0.5625, 0.4375],
        [0.8125, 0.6875, 0.4375, 0.0625, 0.9375],
        [0.5625, 0.4375, 0.1875, 0.8125, 0.6875],
        [0.0625, 0.9375, 0.6875, 0.3125, 0.1875],
        [0.0938, 0.4688, 0.8438, 0.4062, 0.2812],
        [0.5938, 0.9688, 0.3438, 0.9062, 0.7812],
        [0.8438, 0.2188, 0.0938, 0.1562, 0.5312],
        [0.3438, 0.7188, 0.5938, 0.6562, 0.0312],
        [0.4688, 0.0938, 0.4688, 0.2812, 0.6562]])

In [19]:
opt_func(torch.tensor([[1., 2.], [1., 1.]]))

tensor([2.2361, 1.4142])

In [12]:
X, y

(tensor([[ 0.7000,  0.7000,  0.7000,  0.7000,  0.7000],
         [ 3.2000, -1.8000,  3.2000, -1.8000,  3.2000],
         [-1.8000,  3.2000, -1.8000,  3.2000, -1.8000],
         [-0.5500, -0.5500,  1.9500, -3.0500,  4.4500],
         [ 4.4500,  4.4500, -3.0500,  1.9500, -0.5500],
         [ 1.9500, -3.0500, -0.5500, -0.5500, -3.0500],
         [-3.0500,  1.9500,  4.4500,  4.4500,  1.9500],
         [-2.4250, -1.1750, -1.1750,  2.5750,  1.3250],
         [ 2.5750,  3.8250,  3.8250, -2.4250, -3.6750],
         [ 5.0750, -3.6750,  1.3250,  5.0750, -1.1750],
         [ 0.0750,  1.3250, -3.6750,  0.0750,  3.8250],
         [-1.1750, -2.4250,  5.0750,  1.3250,  0.0750],
         [ 3.8250,  2.5750,  0.0750, -3.6750,  5.0750],
         [ 1.3250,  0.0750, -2.4250,  3.8250,  2.5750],
         [-3.6750,  5.0750,  2.5750, -1.1750, -2.4250],
         [-3.3625,  0.3875,  4.1375, -0.2375, -1.4875],
         [ 1.6375,  5.3875, -0.8625,  4.7625,  3.5125],
         [ 4.1375, -2.1125, -3.3625, -2.7375,  1

In [9]:
state_dict = None
for i in range(100):
    GP = lambda X, y, noise, borders: CustomCylindricalGP(X, y.view(-1, 1), noise, borders)
    acquisition = lambda gp, y: ExpectedImprovement(gp, y.min(), maximize=False)
    objective = lambda x: opt_func(x)
    X, y, gp = bo_step(
        X,
        y.view(-1, 1),
        noise=torch.zeros_like(y), # placeholder, not used
        objective=objective,
        bounds=borders.t(),
        GP=GP,
        acquisition=acquisition,
        q=1,
        state_dict=state_dict
    )
    state_dict = gp.state_dict()
    clear_output()
    print("Last measurement", X[-1], y[-1])
    print("Best measurement", X[y.argmin()], y.min())

Last measurement tensor([0.9519, 1.3770, 0.7980, 1.0888, 0.2234]) tensor([2.1620])
Best measurement tensor([-0.0048,  0.0826,  0.0454,  0.2058,  0.2111]) tensor(0.3096)


In [20]:
state_dict

OrderedDict([('likelihood.noise_covar.raw_noise', tensor([0.0373])),
             ('likelihood.noise_covar.noise_prior.concentration',
              tensor(1.1000)),
             ('likelihood.noise_covar.noise_prior.rate', tensor(0.0500)),
             ('mean_module.constant', tensor([5.0975])),
             ('covar_module.raw_outputscale', tensor(-1.0397)),
             ('covar_module.base_kernel.raw_angular_weights',
              tensor([13.5099,  1.0031, -3.6613, -3.2481])),
             ('covar_module.base_kernel.raw_alpha', tensor([6.3350])),
             ('covar_module.base_kernel.raw_beta', tensor([1.4034])),
             ('covar_module.base_kernel.radial_base_kernel.raw_lengthscale',
              tensor([[-0.3850]]))])

In [21]:
X

tensor([[ 0.7000,  0.7000,  0.7000,  0.7000,  0.7000],
        [ 3.2000, -1.8000,  3.2000, -1.8000,  3.2000],
        [-1.8000,  3.2000, -1.8000,  3.2000, -1.8000],
        [-0.5500, -0.5500,  1.9500, -3.0500,  4.4500],
        [ 4.4500,  4.4500, -3.0500,  1.9500, -0.5500],
        [ 1.9500, -3.0500, -0.5500, -0.5500, -3.0500],
        [-3.0500,  1.9500,  4.4500,  4.4500,  1.9500],
        [-2.4250, -1.1750, -1.1750,  2.5750,  1.3250],
        [ 2.5750,  3.8250,  3.8250, -2.4250, -3.6750],
        [ 5.0750, -3.6750,  1.3250,  5.0750, -1.1750],
        [ 0.0750,  1.3250, -3.6750,  0.0750,  3.8250],
        [-1.1750, -2.4250,  5.0750,  1.3250,  0.0750],
        [ 3.8250,  2.5750,  0.0750, -3.6750,  5.0750],
        [ 1.3250,  0.0750, -2.4250,  3.8250,  2.5750],
        [-3.6750,  5.0750,  2.5750, -1.1750, -2.4250],
        [-3.3625,  0.3875,  4.1375, -0.2375, -1.4875],
        [ 1.6375,  5.3875, -0.8625,  4.7625,  3.5125],
        [ 4.1375, -2.1125, -3.3625, -2.7375,  1.0125],
        [-